In [41]:
import os
import pandas as pd

In [42]:
transitive_closure = pd.read_csv('/home/dc925/project/data/snomed/transitive_closure.txt', sep='\t')
mrconso = pd.read_csv('/home/dc925/project/data/snomed/MRCONSO.RRF', sep='|', header=None)

/home/dc925/miniconda3/envs/pytorch/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [45]:
transitive_closure['superTypeId'] = transitive_closure['superTypeId'].apply(lambda x: str(x))
transitive_closure['subTypeId'] = transitive_closure['subTypeId'].apply(lambda x: str(x))

In [46]:
transitive_closure[transitive_closure.duplicated()];

In [47]:
mrconso;

In [64]:
df = mrconso[[0,13]]
df.columns  = ['CUI', 'SCUI']
df = df[-df.duplicated()]
df['SCUI'] = df['SCUI'].apply(lambda x: str(x))

In [65]:
df

,CUI,SCUI
0,C0000039,102735002
2,C0000052,58488005
7,C0000097,285407008
9,C0000102,13579002
14,C0000163,112116001
...,...,...
1479246,C4759626,11309241000119105
1479249,C4759627,11313401000119108
1479252,C4759628,11313361000119104
1479255,C4759629,447562003


In [67]:
snomed_concepts_in_mrconso = set(df['SCUI'])

In [68]:
snomed_concepts_in_mrconso;

In [69]:
snomed_source_concepts= set(transitive_closure['superTypeId']) | set(transitive_closure['subTypeId'])

In [70]:
len(snomed_concepts_in_mrconso)

473759

In [71]:
len(snomed_source_concepts)

357405

In [74]:
intersected_scui = snomed_concepts_in_mrconso & snomed_source_concepts
len(intersected_scui)

353045

In [75]:
transitive_closure = transitive_closure[(transitive_closure['superTypeId'].isin(intersected_scui)) & (transitive_closure['subTypeId'].isin(intersected_scui))]

In [76]:
df = df[df['SCUI'].isin(intersected_scui)]

In [77]:
scui2cui = df.set_index('SCUI').to_dict()['CUI']

In [82]:
transitive_closure_final = pd.concat([transitive_closure['subTypeId'].apply(lambda x: scui2cui[x]), transitive_closure['superTypeId'].apply(lambda x: scui2cui[x])], axis=1)




In [84]:
transitive_closure_final['RELA'] = 'isa'

In [86]:
transitive_closure_final = transitive_closure_final[['subTypeId','RELA','superTypeId']]

In [89]:
transitive_closure_final.to_csv('transitive_closure_full.txt', sep='\t', header=None, index=None)